In [1]:
import numpy as np
import torch
import psutil
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import time
import matplotlib.pyplot as plt
import random
import sys
import pickle
from torch.optim import SGD

In [2]:
sys.path.insert(1, '/home/ubuntu/Intents-Analysis/Analysis')
#sys.path.insert(1, '/Users/manjugupta/Desktop/CMU_Courses/Intents/getting_intents/Analysis')

In [3]:
from get_vocab import load_data, get_vocab
from get_frequency import get_frequency

In [4]:
#Check if cuda is available
cuda = torch.cuda.is_available()
print('CUDA is', cuda)

num_workers = 8 if cuda else 0

print(num_workers)

CUDA is True
8


In [5]:
##Needed Functions
def load_data(filename):
    a_file = open(filename, "rb")
    output = pickle.load(a_file)
    a_file.close()
    return output


def create_vocabulary(train_file):
    '''This function creates an indexed vocabulary dictionary from the training file'''
    
    vocab, _ = get_vocab(1, train_file)
    
    phone_to_idx = {'unk': 1}#Padding indx = 0, unkown_idx = 1, indexing starts from 2
    for i, phone in enumerate(vocab):
        phone_to_idx[phone] = i + 2
        
    return phone_to_idx

In [6]:
class MyDataset(Dataset):
    def __init__(self, data_file, intent_labels, phone_to_idx):
        data = load_data(data_file)
        self.all_data = []
        
        for intent in data:
            for utterance in data[intent]:
                utterance_to_idx = []
                
                for phone in utterance:
                    if phone not in phone_to_idx:
                        phone = 'unk'
    
                    utterance_to_idx.append(phone_to_idx[phone])
                
                self.all_data.append([utterance_to_idx, intent_labels[intent]])
            
    def __len__(self):
        return len(self.all_data)

    def __getitem__(self,index):
        input_vector = self.all_data[index][0]
        label = self.all_data[index][1]

        return input_vector, label

In [7]:
def collate_indic(tuple_lst):

    x_lst = [x[0] for x in tuple_lst]
    y_lst = [x[1] for x in tuple_lst]

    # collate x
    B = len(tuple_lst)#Number of training samples
    T = max(len(x) for x in x_lst)#Max length of a sentence

    # x values
    x = torch.zeros([B, T], dtype=torch.int64)
    lengths = torch.zeros(B, dtype=torch.int64)

    for i, x_np in enumerate(x_lst):
        lengths[i] = len(x_np)
        x[i,:len(x_np)] = torch.tensor(x_np)

    # collate y
    y = torch.zeros([B, 6])
    for i, y_label in enumerate(y_lst):
        y[i][y_label] = 1
        
    ids = torch.argsort(lengths, descending=True)

    return x[ids], lengths[ids], y[ids]

In [8]:
#Defining constants and labels
intent_labels = {'movie-tickets':0, 'auto-repair':1, 'restaurant-table':2, 'pizza-ordering':3, 'uber-lyft':4, 'coffee-ordering':5}
train_language = 'gujarati_marathi_bengali_hindi_05'
test_language = 'hindi'

#Loading data
train_file = '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/3_lang_variations/taskmaster_training_' + train_language + '.pkl'
test_file = '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/1_language/taskmaster_testing_' + test_language + '.pkl'

#create vocabulary and phone_to_idx
phone_to_idx = create_vocabulary(train_file)
print(len(phone_to_idx))

FileNotFoundError: [Errno 2] No such file or directory: '/home/ubuntu/Intents-Analysis/TaskMasterData/Get_Phones_Combos/3_lang_variations/taskmaster_training_hindi.pkl'

In [10]:
train_dataset = MyDataset(train_file, intent_labels, phone_to_idx)
train_loader_args = dict(shuffle=True, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=True, batch_size=32)
train_loader = DataLoader(train_dataset, **train_loader_args, collate_fn=collate_indic)

test_dataset = MyDataset(test_file, intent_labels, phone_to_idx)
test_loader_args = dict(shuffle=False, batch_size=128, num_workers=num_workers, pin_memory=True) if cuda\
                    else dict(shuffle=False, batch_size=1)
valid_loader = DataLoader(test_dataset, **test_loader_args, collate_fn=collate_indic)

In [11]:
class RNNClassifier(nn.Module):
    def __init__(self, vocab_size=68, embed_size=128, hidden_size=128, label_size=6):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)

        self.cnn  = nn.Conv1d(embed_size, embed_size, kernel_size=3, padding=1)
        self.cnn2 = nn.Conv1d(embed_size, embed_size, kernel_size=5, padding=2)
        self.cnn3 = nn.Conv1d(embed_size, embed_size, kernel_size=7, padding=3)

        self.batchnorm = nn.BatchNorm1d(embed_size*3)

        self.lstm = nn.LSTM(embed_size*3, hidden_size, num_layers=1)
        self.linear = nn.Linear(hidden_size, label_size)

    def forward(self, x, lengths):
        """
        padded_x: (B,T) padded LongTensor
        """

        # B,T,H
        input = self.embed(x)

        # (B,T,H) -> (B,H,T)
        input = input.transpose(1,2)

        cnn_output = torch.cat([self.cnn(input), self.cnn2(input), self.cnn3(input)], dim=1)

        # (B,H,T)
        input = F.relu(self.batchnorm(cnn_output))

        input = input.transpose(1,2)

        pack_tensor = nn.utils.rnn.pack_padded_sequence(input, lengths, batch_first=True)

        output, (hn, cn) = self.lstm(pack_tensor)

        #output, _ = nn.utils.rnn.pad_packed_sequence(output, batch_first=True)

        #output = torch.cat([hn[0], hn[1]], dim=1)
        logits = self.linear(hn[0])

        return logits

In [14]:
model = RNNClassifier()
opt = optim.Adam(model.parameters(), lr = 0.001)
#opt = SGD(model.parameters(), lr=0.05)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

RNNClassifier(
  (embed): Embedding(68, 128)
  (cnn): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (cnn2): Conv1d(128, 128, kernel_size=(5,), stride=(1,), padding=(2,))
  (cnn3): Conv1d(128, 128, kernel_size=(7,), stride=(1,), padding=(3,))
  (batchnorm): BatchNorm1d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(384, 128)
  (linear): Linear(in_features=128, out_features=6, bias=True)
)

In [15]:
print(train_language, test_language)
max_acc = 0

for i in range(1000):
    #print("epoch ", i)
    loss_accum = 0.0
    batch_cnt = 0

    acc_cnt = 0
    err_cnt = 0

    model.train()
    start_time = time.time()
    for batch, (x, lengths, y) in enumerate(train_loader):

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        opt.zero_grad()
        
        logits = model(x, lengths)

        loss = nn.BCEWithLogitsLoss()(logits, y)
        loss_score = loss.cpu().item()

        loss_accum += loss_score
        batch_cnt += 1
        loss.backward()
        opt.step()

        out_val, out_indices = torch.max(logits, dim=1)
        target_val, tar_indices = torch.max(y, dim=1)

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    print("train acc: ", acc_cnt/(err_cnt+acc_cnt), " train loss: ", loss_accum / batch_cnt, '--time:', time.time() - start_time)

    model.eval()
    acc_cnt = 0
    err_cnt = 0

    #start_time = time.time()
    for x, lengths, y in valid_loader:

        x = x.to(device)
        lengths = lengths.to(device)
        y = y.to(device)
        
        logits = model(x, lengths)

        out_val, out_indices = torch.max(logits, dim=1)
        target_val, tar_indices = torch.max(y, dim=1)

        for i in range(len(out_indices)):
            if out_indices[i] == tar_indices[i]:
                acc_cnt += 1
            else:
                err_cnt += 1

    current_acc = acc_cnt/(err_cnt+acc_cnt)
    if current_acc > max_acc:
        max_acc = current_acc
                
    print("validation: ", current_acc, '--max', max_acc, '--time:', time.time() - start_time)

gujarati_marathi_bengali_hindi_05 hindi
train acc:  0.25178389398572887  train loss:  0.4556196785491446 --time: 5.27141809463501
validation:  0.17333333333333334 --max 0.17333333333333334 --time: 5.917364835739136
train acc:  0.2969758749575263  train loss:  0.43075917596402374 --time: 5.191982269287109
validation:  0.17 --max 0.17333333333333334 --time: 5.925041675567627
train acc:  0.3367312266394835  train loss:  0.4192426969175753 --time: 4.926982164382935
validation:  0.19666666666666666 --max 0.19666666666666666 --time: 5.576266527175903
train acc:  0.4274549779136935  train loss:  0.3968094276345294 --time: 4.85190486907959
validation:  0.2633333333333333 --max 0.2633333333333333 --time: 5.662023305892944
train acc:  0.5225959904858988  train loss:  0.36370921782825305 --time: 4.938372611999512
validation:  0.3333333333333333 --max 0.3333333333333333 --time: 5.651646137237549
train acc:  0.6041454298335033  train loss:  0.32105090436728106 --time: 3.729836940765381
validation: 

train acc:  0.9976214746856948  train loss:  0.005482553382930549 --time: 5.033552646636963
validation:  0.75 --max 0.75 --time: 5.817176103591919
train acc:  0.9983010533469249  train loss:  0.0037610126892104745 --time: 5.18383526802063
validation:  0.7333333333333333 --max 0.75 --time: 5.92573618888855
train acc:  0.9921848453958546  train loss:  0.00956611810049609 --time: 5.449359178543091
validation:  0.7133333333333334 --max 0.75 --time: 6.21384072303772
train acc:  0.9959225280326198  train loss:  0.006902260503367237 --time: 5.315237283706665
validation:  0.7466666666666667 --max 0.75 --time: 6.104578495025635
train acc:  0.9983010533469249  train loss:  0.0031130389340788774 --time: 5.385574579238892
validation:  0.75 --max 0.75 --time: 6.106876850128174
train acc:  0.9996602106693849  train loss:  0.0017404554199184413 --time: 4.92876672744751
validation:  0.7566666666666667 --max 0.7566666666666667 --time: 5.661174297332764
train acc:  1.0  train loss:  0.001172678593201967

validation:  0.7833333333333333 --max 0.7833333333333333 --time: 5.724406003952026
train acc:  1.0  train loss:  0.000264275209148131 --time: 5.043195486068726
validation:  0.7833333333333333 --max 0.7833333333333333 --time: 5.737087249755859
train acc:  1.0  train loss:  0.00025972987603618884 --time: 5.087347030639648
validation:  0.78 --max 0.7833333333333333 --time: 5.833815097808838
train acc:  1.0  train loss:  0.00025524182685488915 --time: 4.937372922897339
validation:  0.7766666666666666 --max 0.7833333333333333 --time: 5.584127902984619
train acc:  1.0  train loss:  0.0002483834010174336 --time: 4.810994386672974
validation:  0.78 --max 0.7833333333333333 --time: 5.548080682754517
train acc:  1.0  train loss:  0.00024272633891087025 --time: 5.104565382003784
validation:  0.78 --max 0.7833333333333333 --time: 5.822178363800049
train acc:  1.0  train loss:  0.00024088159325006217 --time: 5.008148431777954
validation:  0.7833333333333333 --max 0.7833333333333333 --time: 5.673076

train acc:  1.0  train loss:  0.00010406677630152959 --time: 4.840366363525391
validation:  0.78 --max 0.7833333333333333 --time: 5.502370357513428
train acc:  1.0  train loss:  0.00010298480281009056 --time: 4.673288345336914
validation:  0.78 --max 0.7833333333333333 --time: 5.4128100872039795
train acc:  1.0  train loss:  0.00010100611088210312 --time: 4.46805477142334
validation:  0.78 --max 0.7833333333333333 --time: 5.217271327972412
train acc:  1.0  train loss:  9.960707266936484e-05 --time: 4.594513416290283
validation:  0.78 --max 0.7833333333333333 --time: 5.353996276855469
train acc:  1.0  train loss:  9.83381994402684e-05 --time: 5.0203163623809814
validation:  0.7833333333333333 --max 0.7833333333333333 --time: 5.733248710632324
train acc:  1.0  train loss:  9.612293449564554e-05 --time: 5.131403923034668
validation:  0.78 --max 0.7833333333333333 --time: 5.650821924209595
train acc:  1.0  train loss:  9.493492625947313e-05 --time: 5.4568352699279785
validation:  0.7833333

validation:  0.79 --max 0.7933333333333333 --time: 5.954129219055176
train acc:  1.0  train loss:  4.5918592545406327e-05 --time: 4.927094459533691
validation:  0.79 --max 0.7933333333333333 --time: 5.727493524551392
train acc:  1.0  train loss:  4.498539435545631e-05 --time: 5.023127317428589
validation:  0.7866666666666666 --max 0.7933333333333333 --time: 5.805574417114258
train acc:  1.0  train loss:  4.463645872538505e-05 --time: 4.888556480407715
validation:  0.7833333333333333 --max 0.7933333333333333 --time: 5.525676488876343
train acc:  1.0  train loss:  4.40406996779569e-05 --time: 4.794149398803711
validation:  0.79 --max 0.7933333333333333 --time: 5.452211141586304
train acc:  1.0  train loss:  4.332711234936774e-05 --time: 5.065409183502197
validation:  0.78 --max 0.7933333333333333 --time: 5.844436883926392
train acc:  1.0  train loss:  4.2575862449745685e-05 --time: 4.925143718719482
validation:  0.78 --max 0.7933333333333333 --time: 5.671648979187012
train acc:  1.0  tra

KeyboardInterrupt: 